# Tutorial BuildME

This tutorial shows the most important functionalities of the BuildME framework using a mix of interactive code cells and text cells. The tutorial walks the user through the most important steps shown also in the `main.py` file of the framework. 

During the tutorial, we: 1) select a building archetype and set up the framework, 2) simulate its energy demand using Energy+, 3) calculate its material demand, 4) process the results. 

## 1. Setting up the framework

Before we can proceed, some initial setup needs to be performed: Python dependencies and Energy+ need to be installed, and the paths in `BuildME/settings.py` might need to be updated. These steps and other potential setup issues are described [here](https://github.com/nheeren/BuildME/blob/master/docs/setup.md).

### Imports

We need to import the BuildME framework (and the `os` package which allows us to change the current working directory).

In [ ]:
import os

if os.path.basename(os.getcwd()) != 'BuildME':
    os.chdir('../..')

from BuildME import pre, simulate

### Selection of archetypes

As the next step, we need to define the type of building we want to simulate. We can do that by creating a variable `debug_combinations`. This variable should contain information such as region, occupation and other aspects. The possible combinations of the different aspects are listed in the `combinations` variable in `settings.py`. 

Let's say we want to simulate a single family house (**SFH**) located in the **US**. We assume that the building is built without extra requirements on resource efficiency (**RES0**) and that the cooling demand is fulfilled solely using the **HVAC** system. We choose mixed humid climate such as in Baltimore (climate region **4A**) in the reference year **2015**. We are interested in how the energy standard influences the energy use in the building, so we select four different ones: **standard**, **non-standard** (below standard), **efficient** and **ZEB**.

The keywords marked in bold are the categories we insert to our `debug_combinations` dictionary, as shown below. Using the dictionary, the framework can proceed, assuming that the required archetype file (.idf extension) and weather file (.epw extension) are present. 

The archetype files are provided by the framework and can be found in folder `BuildME\data\archetype\USA\`. The weather files need to be created by the user using dedicated software, e.g. Meteonorm, and can be found in folder `BuildME\data\climate\meteonorm71\**reference year**\`. The names of the weather file for each climate are specified in the `climate_stations` variable in `settings.py`. If the weather file is not found, BuildME uses a dummy weather file for New York, NY (climate region 4A), [made available by the U.S. Department of Energy](https://www.energycodes.gov/prototype-building-models).

In [ ]:
debug_combinations = {
    'USA':
        {'occupation': ['SFH'],
         'energy standard': ['non-standard', 'standard', 'efficient', 'ZEB'],
         'RES': ['RES0'],
         'climate_region': ['4A'],
         'climate_scenario': ['2015'],
         'cooling': ['HVAC']}}

### Checking EnergyPlus compatibility

The next step is to check the EnergyPlus version - the one installed by the user locally should match the one that is used by BuildME. 

In [ ]:
pre.validate_ep_version()

### Creating MMV variants (optional)

If the user has selected 'MMV' as the cooling type, then the next step is to create a MMV variant out of the archetype that was chosen for simulation. If the cooling type for every building combination is 'HVAC', this simulation step could be skipped.

In [ ]:
pre.create_mmv_variants(comb=debug_combinations)

### Preparing the files for simulations


Based on the previously defined archetype (variable `debug_combinations`), we create the files to be used in energy and/or material simulations. The first step is to create an array `simulation_files` with information about each building instance to be simulated. Each element of this array includes a list with necessary information about a building, such as the location of the .EPW climate file or the IDF building file. 

In addition, we create a variable `run` which is a unique identifier of this model run (created based on the timestamp).

In [ ]:
simulation_files, run = simulate.create_combinations(debug_combinations)

The next step ensures that folders created in previous model runs do not interfere with the current simulation workflow. In case such folders are identified, the `nuke_folders()` function deletes them.

In [ ]:
simulate.nuke_folders(simulation_files, run)  # deletes only the folder with the case you try to simulate

Finally, based on the information stored in the `simulation_files` variable, the `copy_scenario_files()` function creates new folders in the `tmp` folder. 

The folders are then populated with weather files (.EPW) and building files (.IDF) chosen depending on the archetypes and climates selected in the 'debug_combinations' step. 

This step includes an essential part of the BuildME framework: substituting the so-called *replacement* strings. Inside the `copy_scenario_files()` function, we find `apply_obj_name_change()` routine applied to energy standard and resource efficiency scenarios. The routine substitutes the '-en-std-replaceme' and '-res-replaceme' strings included in the names of some materials by strings specific to the simulated building (e.g. 'standard' and 'RES0' like in our example). This way, BuildME automatizes the implementation of various energy standards which use e.g. insulation layers of different thickness.

In [ ]:
simulate.copy_scenario_files(simulation_files, run)

After performing the step described above, the respective folders contain the EPW and IDF files necessary for the simulation. The IDF file called 'in.idf' is ready for simulation, e.g. all the 'replaceme' strings have been substituted. This means that the file could also be used to manually simulate energy demand using EnergyPlus GUI called EP-Launch.exe or similar. Please note that if the `copy_scenario_files()` function was to be omitted, then the 'replaceme' strings would *not* be substituted and the simulation would fail because these names would not match any constructions defined in the IDF file.

## 2. Simulating energy demand

At this point, we can perform the energy simulation using EnergyPlus. Please note that this step takes more time to execute,  depending on the chosen archetype and cooling strategy; it can take between around 20 seconds (e.g., for SFH archetype with HVAC cooling) and more than 15 minutes (e.g., MFH archetype with MMV cooling). After the simulation is done, a prinout message appears below, indicating the number of seconds it took to simulate one building (= one iteration) e.g., 22.26s/it.

In [ ]:
simulate.calculate_energy()

After performing the energy simulation, the respective folders are filled with simulation files (typically more than 30). The most important of these are:
- `eplusout.err` which informs us about critical errors (in case the simulation did not succeed) or warnings;
- `eplusout.csv` which includes variable values, as listed in the *Output:Variable* object of the IDF file;
- `eplusmtr.csv` which includes metered values, as listed in the *Output:Meter* object of the IDF file.

## 3. Simulating material demand

To calculate the amount of material used in the building, BuildME performs a number of operations. First, it lists the constructions used in the building: walls, floors, ceilings, roof, doors, window glazing. After listing all the layers of materials used in these constructions, it adds up their volumes, which are then converted to mass. In addition, some surrogate structures are created, such as a basement and extra internal walls (corresponding to internal partitions; typically not represented in Energy+, which limits internal walls to those defining building zones).

The materials are then grouped into categories such as 'concrete', 'brick', 'paper and cardboard' etc., as indicated by the `odym_materials` variable found in the `settings.py` file.

After performing this step, a file `materials.csv` appears in the respective folder. The file contains the material categories and their mass (kg) contained in the entire building. The floor area of the building (m<sup>2</sup>) is also given (`floor_area_wo_basement`), and can be used to calculate the material intensity.

In [ ]:
simulate.calculate_materials()

## 4. Processing the results

After all the simulations have been performed, the results can be processed and structured. 

The first post-simulation step requires collection of energy and material data. For each building combination, file `energy_demand.csv` is created. It lists energy use categories (heating, cooling, lighting, equipment) and their energy use (in joules, for the entire building). 

Dictionaries `res_energy` and `res_mat` are created to contain the energy and material demand information for all the building combinations. 

In [ ]:
res_energy = simulate.collect_energy()
res_mat = simulate.load_material()

### Data visualization - table

At this point, the results for all building combinations can be visualized using a few simple data manipulation steps performed with the use of the `pandas` and `numpy` library. Below, we can see the energy (MJ) and material demand (kg) for 1 m2 of the building.

In [ ]:
import pandas as pd
import numpy as np
df_mat = pd.DataFrame(data=res_mat).T
area = df_mat['floor_area_wo_basement']
df_mat = df_mat.div(area, axis=0)
df_mat = df_mat.drop(['floor_area_wo_basement', 'footprint_area'], axis=1)
df_mat = df_mat.drop(['total_mat'], axis=1)

df_ene = pd.DataFrame(data=res_energy).T
df_ene2 = pd.DataFrame(data=res_energy)
df_ene = df_ene.div(area, axis=0)
df_ene = df_ene.rename(columns={v: v.split(':')[0] for v in list(df_ene.columns)})
split = [v.split('_') for v in list(df_ene.index)]
categories = ['region', 'occupation', 'energy standard','RES', 'climate_region', 'climate_scenario', 'cooling']
chosen_categories = []
for i,category in enumerate(categories):
    items = [item[i] for item in split]
    if not all(ele == items[0] for ele in items):
        df_ene[category] = items
        df_mat[category] = items
        chosen_categories.append(category)
df_ene = df_ene.set_index(chosen_categories)
df_mat = df_mat.set_index(chosen_categories)

In [ ]:
df_ene

In [ ]:
df_mat

### Data visualization - simple

We can also plot the results using the 'matplotlib' library. For easier comparison, the results are converted to relative values. Optionally, we can filter the dataframe and only display some part of the results, which is done by slicing the dataframe's multiindex. The filter can be customized, depending on the combinations of archetypes chosen for the simulation. The multiindex names and values can be found by printing `df_ene.index`.

In [ ]:
# print(df_ene.index)
# df_ene = df_ene[np.in1d(df_ene.index.get_level_values(0), ['standard'])] # optional customizable filter
# df_mat = df_mat[np.in1d(df_mat.index.get_level_values(0), ['standard'])] # optional customizable filter
import matplotlib.pyplot as plt
(df_ene/df_ene.max()).plot.bar(rot=30)
plt.legend(bbox_to_anchor=(1,1))  
(df_mat/df_mat.max()).plot.bar(rot=30)
plt.legend(bbox_to_anchor=(1,1)) 